In [ ]:
from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1750, 2501, 1)
f.define_scenarios(["high-extension", "high-overshoot", "medium", "medium-overshoot", "low", "verylow", "verylow-overshoot"])
species, properties = read_properties('../data/fair-parameters/species_configs_properties_1.4.1.csv')
f.define_species(species, properties)
f.ch4_method='Thornhill2021'
df_configs = pd.read_csv('../data/fair-parameters/calibrated_constrained_parameters_1.4.1.csv', index_col=0)
f.define_configs(df_configs.index)
f.allocate()

In [ ]:
f.fill_from_csv(
    emissions_file='../data/emissions/extensions_1750-2500.csv',
    forcing_file='../data/forcing/volcanic_solar.csv',
)

In [ ]:
f.fill_species_configs('../data/fair-parameters/species_configs_properties_1.4.1.csv')

In [ ]:
f.override_defaults('../data/fair-parameters/calibrated_constrained_parameters_1.4.1.csv')

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
for scenario in f.scenarios:
    pl.plot(
        f.timebounds,
        (
            f.temperature.sel(scenario=scenario, layer=0)-
            f.temperature.sel(scenario=scenario, layer=0, timebounds=np.arange(1850,1902)).mean(dim='timebounds')
        ).median(dim='config'),
        label=scenario
    )
pl.legend();

In [ ]:
for scenario in f.scenarios:
    pl.plot(
        f.timebounds,
        f.concentration.sel(scenario=scenario, specie="CO2").median(dim='config'),
        label=scenario
    )
pl.legend();

In [ ]:
for scenario in f.scenarios:
    pl.plot(
        f.timebounds,
        f.forcing_sum.sel(scenario=scenario).median(dim='config'),
        label=scenario
    )
pl.legend();